# SpamDetector

https://github.com/udacity/machine-learning/blob/master/projects/practice_projects/naive_bayes_tutorial/Naive_Bayes_tutorial.ipynb

First we will rebuild the Algo Bag of Words in separate Steps to represent the words in the coloums with numbers. Reason: just numbers can handeld by an learning-algorithm.

In [67]:
import pandas as pd

In [68]:
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t', 
                   header=None, 
                   names=['label', 'sms_message'])
print(df.head())

  label                                        sms_message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [69]:
# map ham and spam to 0/1 to deal with it
df['label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
print(df.head())

(5572, 2)
   label                                        sms_message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


In [70]:
print("Convert to lower case")
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
print(documents)
lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

Convert to lower case
['Hello, how are you!', 'Win money, win from home.', 'Call me now.', 'Hello, Call hello you tomorrow?']
['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


In [71]:
print("Remove all punctation")
import string
without_punktation = []
for i in lower_case_documents:
    without_punktation.append(i.translate(str.maketrans('', '', string.punctuation)))
print(without_punktation)

Remove all punctation
['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


In [72]:
print("Split into single words")
words = []
for i in without_punktation:
    words.append(i.split(' '))
print(words)

gezaehlte_Liste = []
import pprint
from collections import Counter
for i in words:
    counts = Counter(i)
    gezaehlte_Liste.append(counts)
pprint.pprint(gezaehlte_Liste)

Split into single words
[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]
[Counter({'hello': 1, 'you': 1, 'how': 1, 'are': 1}),
 Counter({'win': 2, 'from': 1, 'money': 1, 'home': 1}),
 Counter({'now': 1, 'me': 1, 'call': 1}),
 Counter({'hello': 2, 'you': 1, 'tomorrow': 1, 'call': 1})]


# For now we use the ready implemented Bag of Words-Algo

First deal with test-data

In [73]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [74]:
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [75]:
print("Transform the Vector to a Matrix")
print(count_vector.transform(documents))
print("and put it in an Array")
doc_array = count_vector.transform(documents).toarray()
doc_array

Transform the Vector to a Matrix
  (0, 0)	1
  (0, 3)	1
  (0, 5)	1
  (0, 11)	1
  (1, 2)	1
  (1, 4)	1
  (1, 7)	1
  (1, 10)	2
  (2, 1)	1
  (2, 6)	1
  (2, 8)	1
  (3, 1)	1
  (3, 3)	2
  (3, 9)	1
  (3, 11)	1
and put it in an Array


array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

Now the Matrix is transformed into an Array. The frequency of every word ist represented by the count of the numbers. To make it easier to read wie transform it into an DataFrame. There we have headers with the single words.

"DataFrame([data, index, columns, dtype, copy])	Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns)."

In [76]:
freq_matrix = pd.DataFrame(doc_array,
                           columns = count_vector.get_feature_names()
                           )
freq_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


# Create the Training and Testing Set
Now we use the train_test_split from sklearn. First import the needed function from the framework and than use it to store the four data in separate matrices. (Train and test for message and the label)

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['sms_message'],
                                                    df['label'],
                                                    random_state=1,
                                                    test_size = 0.33)
#print("X-Train-Matrix:"); print(X_train); print("X-Test-Matrix:"); print(X_test)
#print("Y-Train-Matrix:"); print(Y_train); print("Y-Test-Matrix:"); print(Y_test)
print('Number of rows in the total set: {}' .format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 3733
Number of rows in the test set: 1839


# Applying Bag of Words to the data


In [78]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)



# Bayes Theorem
First starting with a simple example - calculation the probaility of getting a positiv diabetis testresult


![title](https://raw.githubusercontent.com/udacity/machine-learning/756d918d21551d3c3668ec6ee680947b253d1214/projects/practice_projects/naive_bayes_tutorial/images/bayes_formula.png)

P(A) is the prior probability of A occuring independantly. In our example this is P(D). This value is given to us.
P(B) is the prior probability of B occuring independantly. In our example this is P(Pos).
P(A|B) is the posterior probability that A occurs given B. In our example this is P(D|Pos). That is, the probability of an individual having diabetes, given that, that individual got a positive test result. This is the value that we are looking to calculate.
P(B|A) is the likelihood probability of B occuring, given A. In our example this is P(Pos|D). This value is given to us.

Putting our values into the formula for Bayes theorem we get:
P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)
The probability of getting a positive test result P(Pos) can be calulated using the Sensitivity and Specificity as follows:
P(Pos) = [P(D) * Sensitivity] + [P(~D) * (1-Specificity))]

In [79]:
'''
Instructions:
Calculate probability of getting a positive test result, P(Pos)
'''
#Given facts:
#P(D)
p_diabetis = 0.01
#P{~D}
p_no_diabetis = 0.99
#Sensitivity or P(Pos|D) - fact that test with positive result work fine
p_pos_diabetis = 0.9
#Specificity or P(Neg|~D) - fact that test with negative result work fine
p_negativ_no_diabetis = 0.9

p_pos = (p_diabetis * p_pos_diabetis) + (p_no_diabetis * (1 - p_negativ_no_diabetis))
print('The probability of getting a positive test result P(Pos) is: {}', format(p_pos))

The probability of getting a positive test result P(Pos) is: {} 0.10799999999999998


The probability of an individual having diabetes, given that, that individual got a positive test result:
P(D/Pos) = (P(D) * Sensitivity)) / P(Pos)

The probability of an individual not having diabetes, given that, that individual got a positive test result:
P(~D/Pos) = (P(~D) * (1-Specificity)) / P(Pos)


In [80]:
'''
Instructions:
Compute the probability of an individual having diabetes, given that, that individual got a positive test result.
In other words, compute P(D|Pos).

The formula is: P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)
'''
p_diabetis_pos = (p_diabetis * p_pos_diabetis) / p_pos
print('Probability of an individual having diabetes, given that')
print('that individual got a positive test result is: {}', format(p_diabetis_pos))

Probability of an individual having diabetes, given that
that individual got a positive test result is: {} 0.08333333333333336


In [81]:
'''
Instructions:
Compute the probability of an individual not having diabetes, given that, that individual got a positive test result.
In other words, compute P(~D|Pos).

The formula is: P(~D|Pos) = (P(~D) * P(Pos|~D) / P(Pos)

Note that P(Pos/~D) can be computed as 1 - P(Neg/~D). 

Therefore:
P(Pos/~D) = p_pos_no_diabetes = 1 - 0.9 = 0.1
'''
p_no_diabetis_pos = (p_no_diabetis * (1 - p_negativ_no_diabetis) / p_pos)
print('Probability of an individual not having diabetes,')
print('given that that individual got a positive test result is: {}', p_no_diabetis_pos)

Probability of an individual not having diabetes,
given that that individual got a positive test result is: {} 0.9166666666666666


Congratulations! You have implemented Bayes theorem from scratch. Your analysis shows that even if you get a positive test result, there is only a 8.3% chance that you actually have diabetes and a 91.67% chance that you do not have diabetes. This is of course assuming that only 1% of the entire population has diabetes which of course is only an assumption.

# Calculating Speech-Terms

Let's say that we have two political parties' candidates, 'Jill Stein' of the Green Party and 'Gary Johnson' of the Libertarian Party and we have the probabilities of each of these candidates saying the words 'freedom', 'immigration' and 'environment' when they give a speech:

Probability that Jill Stein says 'freedom': 0.1 ---------> P(F|J)
Probability that Jill Stein says 'immigration': 0.1 -----> P(I|J)
Probability that Jill Stein says 'environment': 0.8 -----> P(E|J)
Probability that Gary Johnson says 'freedom': 0.7 -------> P(F|G)
Probability that Gary Johnson says 'immigration': 0.2 ---> P(I|G)
Probability that Gary Johnson says 'environment': 0.1 ---> P(E|G)

And let us also assume that the probablility of Jill Stein giving a speech, P(J) is 0.5 and the same for Gary Johnson, P(G) = 0.5.

![title](https://github.com/udacity/machine-learning/raw/756d918d21551d3c3668ec6ee680947b253d1214/projects/practice_projects/naive_bayes_tutorial/images/naivebayes.png)



Y ==> the Names Jill and Gary
x1 .. xn ==> the words
by bays - every word is independent

To break this down, we have to compute the following posterior probabilities:

P(J|F,I): Probability of Jill Stein saying the words Freedom and Immigration.
            Using the formula and our knowledge of Bayes' theorem, we can compute this as follows: 
            P(J|F,I) = (P(J) * P(F|J) * P(I|J)) / P(F,I).
            Here P(F,I) is the probability of the words 'freedom' and 'immigration' being said in a speech.

P(G|F,I): Probability of Gary Johnson saying the words Freedom and Immigration.
            Using the formula, we can compute this as follows:
            P(G|F,I) = (P(G) * P(F|G) * P(I|G)) / P(F,I)

In [82]:
'''
Instructions: Compute the probability of the words 'freedom' and 'immigration' being said in a speech, or
P(F,I).

The first step is multiplying the probabilities of Jill Stein giving a speech with her individual 
probabilities of saying the words 'freedom' and 'immigration'. Store this in a variable called p_j_text

The second step is multiplying the probabilities of Gary Johnson giving a speech with his individual 
probabilities of saying the words 'freedom' and 'immigration'. Store this in a variable called p_g_text

The third step is to add both of these probabilities and you will get P(F,I).
'''
p_j = 0.5
p_j_f = 0.1
p_j_i = 0.1
p_j_text = p_j * p_j_f * p_j_i
p_g = 0.5
p_g_f = 0.7
p_g_i = 0.2
p_g_text = p_g * p_g_f * p_g_i
# Compute P(F,I) and store in p_f_i
p_f_i = p_j_text + p_g_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

Probability of words freedom and immigration being said are:  0.075


Now we can compute the probability of P(J|F,I), that is the probability of Jill Stein saying the words Freedom and Immigration and P(G|F,I), that is the probability of Gary Johnson saying the words Freedom and Immigration.

In [83]:
'''
Instructions:
Compute P(J|F,I) using the formula P(J|F,I) = (P(J) * P(F|J) * P(I|J)) / P(F,I) and store it in a variable p_j_fi
and same for Gary
'''
p_j_fi = (p_j * p_j_f * p_j_i) / p_f_i
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

'''
Instructions:
Compute P(G|F,I) using the formula P(G|F,I) = (P(G) * P(F|G) * P(I|G)) / P(F,I) and store it in a variable p_g_fi
'''
p_g_fi = (p_g * p_g_f * p_g_i) / p_f_i
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668
The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


Our analysis shows that there is only a 6.6% chance that Jill Stein of the Green Party uses the words 'freedom' and 'immigration' in her speech as compard the the 93.3% chance for Gary Johnson of the Libertarian party.

Applying this to our problem of classifying messages as spam, the Naive Bayes algorithm looks at each word individually and not as associated entities with any kind of link between them. In the case of spam detectors, this usually works as there are certain red flag words which can almost guarantee its classification as spam, for example emails with words like 'viagra' are usually classified as spam.

# Finally we do it with sklearn



In [84]:
'''
Instructions:

We have loaded the training data into the variable 'training_data' and the testing data into the 
variable 'testing_data'.

Import the MultinomialNB classifier and fit the training data into the classifier using fit(). Name your classifier
'naive_bayes'. You will be training the classifier using 'training_data' and y_train' from our split earlier. 
'''
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(training_data, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [85]:
'''
Instructions:
Now that our algorithm has been trained using the training data set we can now make some predictions on the test data
stored in 'testing_data' using predict(). Save your predictions into the 'predictions' variable.
'''
predictions = MNB.predict(testing_data)
# now finished

# Evaluate the model

Accuracy measures how often the classifier makes the correct prediction. It’s the ratio of the number of correct predictions to the total number of predictions (the number of test data points).

Precision tells us what proportion of messages we classified as spam, actually were spam. It is a ratio of true positives(words classified as spam, and which are actually spam) to all positives(all words classified as spam, irrespective of whether that was the correct classificatio), in other words it is the ratio of

[True Positives/(True Positives + False Positives)]

Recall(sensitivity) tells us what proportion of messages that actually were spam were classified by us as spam. It is a ratio of true positives(words classified as spam, and which are actually spam) to all the words that were actually spam, in other words it is the ratio of

[True Positives/(True Positives + False Negatives)]



In [87]:
'''
Instructions:
Compute the accuracy, precision, recall and F1 scores of your model using your test data 'y_test' and the predictions
you made earlier stored in the 'predictions' variable.
'''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(Y_test, predictions)))
print('Precision score: ', format(precision_score(Y_test, predictions)))
print('Recall score: ', format(recall_score(Y_test, predictions)))
print('F1 score: ', format(f1_score(Y_test, predictions)))

Accuracy score:  0.9869494290375204
Precision score:  0.9666666666666667
Recall score:  0.9354838709677419
F1 score:  0.9508196721311476


One of the major advantages that Naive Bayes has over other classification algorithms is its ability to handle an extremely large number of features. In our case, each word is treated as a feature and there are thousands of different words. Also, it performs well even with the presence of irrelevant features and is relatively unaffected by them. The other major advantage it has is its relative simplicity. Naive Bayes' works well right out of the box and tuning it's parameters is rarely ever necessary, except usually in cases where the distribution of the data is known. It rarely ever overfits the data. Another important advantage is that its model training and prediction times are very fast for the amount of data it can handle. All in all, Naive Bayes' really is a gem of an algorithm!
Congratulations! You have succesfully designed a model that can efficiently predict if an SMS message is spam or not!
Thank you for learning with us!